In [ ]:
# Import necessary modules
from draft import Draft

# Set parameters
draft_format = 'Snake'

# Initialize draft
d = Draft(draft_format=draft_format)
d.pre_draft()
d.draft()





ROUND 1


Andrew Bender, you're on the clock!
